In [1]:
import argparse
import os


from dotenv import load_dotenv
load_dotenv()

import wandb
from transformers import RobertaConfig,AutoConfig
from transformers import RobertaForMaskedLM
import torch
import traceback
import logging

from datasets import load_dataset,load_from_disk
import util
import time
import util
import torch 

from util import get_checkpoints_hub
from util import DeterministicDataCollatorForLanguageModeling

from multiprocessing import Pool, Queue, Manager

import math
import numpy as np
from transformers import RobertaTokenizerFast,GPT2TokenizerFast, DataCollatorForLanguageModeling,LlamaForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
 

In [2]:
device = "cuda"

In [169]:
results = load_from_disk("./results_confidence/triviaqa-test-tulu3-query").to_pandas()

In [176]:
results = load_from_disk("./results_confidence/triviaqa-test-tulu3-query").to_pandas()
confidence_cols = ['response_query','confidence_query']#[key for key in results.columns if "_query" in key and "_influence" not in key]
for key in confidence_cols:
        results[key] = results[key].apply(lambda x: [list(a.values())[-1]["text"] for a in eval(x)])
      

In [177]:
confidence_cols

['response_query', 'confidence_query']

In [182]:
list(zip(confidence_cols, (c + "_influence" for c in confidence_cols)))[0]

('response_query', 'response_query_influence')

In [184]:
len(results["response_query"])

10

In [185]:
len(results["response_query_influence"])

10

In [186]:
list(list(zip(confidence_cols, (c + "_influence" for c in confidence_cols)))[0])

['response_query', 'response_query_influence']

In [183]:
results.explode(list(list(zip(confidence_cols, (c + "_influence" for c in confidence_cols)))[0]))

ValueError: columns must have matching element counts

In [187]:
results

,question,target,response,confidence,question_query,response_query,confidence_query,qa_query,pc_query,confidence_query_influence,response_query_influence
0,"If ultimate means last, what word means second...",Penult,penultimate,0.99,[],[Develop a Python algorithm that identifies th...,[A large-scale industrial farmer is considerin...,[],[],"[-0.54931640625, 4.2265625, 14.1875, 1.2246093...","[2.04296875, 9.7265625, -2.326171875, 7.097656..."
1,"""Who was the villain in the movie """"Thor""""?""",Loptr,Loki,0.95,[],"[In this task, you will be shown a short story...",[A cremation specialist is tasked with optimiz...,[],[],"[5.875, 0.3681640625, 5.84765625, 4.12890625, ...","[22.046875, -2.3359375, 8.84375, 1.115234375, ..."
2,In which U.S. State is the University of Harva...,Masachusetts,Massachusetts,0.10,[],[Turn the facts in this article into a series ...,"[A language professor, who deeply respects his...",[],[],"[4.6796875, 8.5234375, 8.6484375, 12.4765625, ...","[1.8955078125, 1.45703125, 2.197265625, 3.2128..."
3,With what group are Bobby Seale and Huey P. Ne...,Black Panther Party,black panthers,0.90,[],[Is this product review positive? Title: Waste...,[A politician who is staunchly against major r...,[],[],"[0.0009679794311523438, 2.80859375, 2.43164062...","[13.3203125, -10.0234375, 0.00457763671875, 0...."
4,"What is the name of 11 Roman Emperors, the fir...",Byzantine Constantine,Byzantine Emperors,0.90,[],[In what ways did Kyiv see itself as heirs to ...,[A politician who is staunchly against major r...,[],[],"[0.00033974647521972656, 3.787109375, 2.103515...","[-1.01953125, 7.66015625, -26809761792.0, 3.07..."
5,Erotology is the study of what?,The word sex,sexual practices and customs,0.90,[],[],[A politician who is staunchly against major r...,[],[],"[0.0007748603820800781, 3.2890625, 2.544921875...","[-2.924177953162028e-27, 4.5777618232563124e-41]"
6,"Grizzled Giant, Southern Flying and Provost’s ...",Skiouros,birds,0.90,[],"[Dr. Finch, a wildlife ecologist specializing ...",[A politician who is staunchly against major r...,[],[],"[0.0014677047729492188, 1.7880859375, 2.763671...","[0.658203125, 5.796875, -4.1171875, -0.5361328..."
7,Who played the character Mrs. Merton on TV?,Caroline Hook,Dawn French,0.99,[],"[Ayat 1: Dalam filem ketiga, ""Fat Lady"" dibint...",[A large-scale industrial farmer is considerin...,[],[],"[-0.0908203125, 4.6484375, 15.90625, 0.7788085...","[28.578125, 3.28125, -1.0787704433333545e-24, ..."
8,Summer Olympic sports are divided into categor...,Freestyle gymnastics,cycling,0.90,[],[Suggest ways to promote cycling culture in th...,[A politician who is staunchly against major r...,[],[],"[0.0014629364013671875, 0.62451171875, 2.05273...","[4.71875, -0.49609375, 24.328125, 8.796875, -0..."
9,The Orinoco river meets the sea on which count...,ISO 3166-1:VE,venezuela,0.99,[],[Write fictional results and standing table fo...,[A large-scale industrial farmer is considerin...,[],[],"[-0.66845703125, 4.7265625, 14.53125, 1.693359...","[2.986328125, -1.5947265625, -96.3125, -0.8222..."


In [49]:
[key for key in results.columns if "_query" in key]

['question_query',
 'response_query',
 'confidence_query',
 'qa_query',
 'pc_query',
 'confidence_query_influence',
 'response_query_influence']

In [42]:
results

,question,target,response,confidence,question_query,response_query,confidence_query,qa_query,pc_query,confidence_query_influence,response_query_influence
0,"If ultimate means last, what word means second...",Penult,penultimate,0.99,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[-0.54931640625, 4.2265625, 14.1875, 1.2246093...","[2.04296875, 9.7265625, -2.326171875, 7.097656..."
1,"""Who was the villain in the movie """"Thor""""?""",Loptr,Loki,0.95,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[5.875, 0.3681640625, 5.84765625, 4.12890625, ...","[22.046875, -2.3359375, 8.84375, 1.115234375, ..."
2,In which U.S. State is the University of Harva...,Masachusetts,Massachusetts,0.10,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[4.6796875, 8.5234375, 8.6484375, 12.4765625, ...","[1.8955078125, 1.45703125, 2.197265625, 3.2128..."
3,With what group are Bobby Seale and Huey P. Ne...,Black Panther Party,black panthers,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0009679794311523438, 2.80859375, 2.43164062...","[13.3203125, -10.0234375, 0.00457763671875, 0...."
4,"What is the name of 11 Roman Emperors, the fir...",Byzantine Constantine,Byzantine Emperors,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.00033974647521972656, 3.787109375, 2.103515...","[-1.01953125, 7.66015625, -26809761792.0, 3.07..."
5,Erotology is the study of what?,The word sex,sexual practices and customs,0.90,[],[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0007748603820800781, 3.2890625, 2.544921875...","[-2.924177953162028e-27, 4.5777618232563124e-41]"
6,"Grizzled Giant, Southern Flying and Provost’s ...",Skiouros,birds,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0014677047729492188, 1.7880859375, 2.763671...","[0.658203125, 5.796875, -4.1171875, -0.5361328..."
7,Who played the character Mrs. Merton on TV?,Caroline Hook,Dawn French,0.99,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[-0.0908203125, 4.6484375, 15.90625, 0.7788085...","[28.578125, 3.28125, -1.0787704433333545e-24, ..."
8,Summer Olympic sports are divided into categor...,Freestyle gymnastics,cycling,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0014629364013671875, 0.62451171875, 2.05273...","[4.71875, -0.49609375, 24.328125, 8.796875, -0..."
9,The Orinoco river meets the sea on which count...,ISO 3166-1:VE,venezuela,0.99,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[-0.66845703125, 4.7265625, 14.53125, 1.693359...","[2.986328125, -1.5947265625, -96.3125, -0.8222..."


In [36]:
results

,question,target,response,confidence,question_query,response_query,confidence_query,qa_query,pc_query,confidence_query_influence,response_query_influence
0,"If ultimate means last, what word means second...",Penult,penultimate,0.99,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[-0.54931640625, 4.2265625, 14.1875, 1.2246093...","[2.04296875, 9.7265625, -2.326171875, 7.097656..."
1,"""Who was the villain in the movie """"Thor""""?""",Loptr,Loki,0.95,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[5.875, 0.3681640625, 5.84765625, 4.12890625, ...","[22.046875, -2.3359375, 8.84375, 1.115234375, ..."
2,In which U.S. State is the University of Harva...,Masachusetts,Massachusetts,0.10,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[4.6796875, 8.5234375, 8.6484375, 12.4765625, ...","[1.8955078125, 1.45703125, 2.197265625, 3.2128..."
3,With what group are Bobby Seale and Huey P. Ne...,Black Panther Party,black panthers,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0009679794311523438, 2.80859375, 2.43164062...","[13.3203125, -10.0234375, 0.00457763671875, 0...."
4,"What is the name of 11 Roman Emperors, the fir...",Byzantine Constantine,Byzantine Emperors,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.00033974647521972656, 3.787109375, 2.103515...","[-1.01953125, 7.66015625, -26809761792.0, 3.07..."
5,Erotology is the study of what?,The word sex,sexual practices and customs,0.90,[],[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0007748603820800781, 3.2890625, 2.544921875...","[-2.924177953162028e-27, 4.5777618232563124e-41]"
6,"Grizzled Giant, Southern Flying and Provost’s ...",Skiouros,birds,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0014677047729492188, 1.7880859375, 2.763671...","[0.658203125, 5.796875, -4.1171875, -0.5361328..."
7,Who played the character Mrs. Merton on TV?,Caroline Hook,Dawn French,0.99,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[-0.0908203125, 4.6484375, 15.90625, 0.7788085...","[28.578125, 3.28125, -1.0787704433333545e-24, ..."
8,Summer Olympic sports are divided into categor...,Freestyle gymnastics,cycling,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0014629364013671875, 0.62451171875, 2.05273...","[4.71875, -0.49609375, 24.328125, 8.796875, -0..."
9,The Orinoco river meets the sea on which count...,ISO 3166-1:VE,venezuela,0.99,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[-0.66845703125, 4.7265625, 14.53125, 1.693359...","[2.986328125, -1.5947265625, -96.3125, -0.8222..."


In [21]:
results.melt(id_vars= [key for key in results.columns if "_query" not in key])

,question,target,response,confidence,variable,value
0,"If ultimate means last, what word means second...",Penult,penultimate,0.99,question_query,[]
1,"""Who was the villain in the movie """"Thor""""?""",Loptr,Loki,0.95,question_query,[]
2,In which U.S. State is the University of Harva...,Masachusetts,Massachusetts,0.10,question_query,[]
3,With what group are Bobby Seale and Huey P. Ne...,Black Panther Party,black panthers,0.90,question_query,[]
4,"What is the name of 11 Roman Emperors, the fir...",Byzantine Constantine,Byzantine Emperors,0.90,question_query,[]
...,...,...,...,...,...,...
65,Erotology is the study of what?,The word sex,sexual practices and customs,0.90,response_query_influence,"[-2.924177953162028e-27, 4.5777618232563124e-41]"
66,"Grizzled Giant, Southern Flying and Provost’s ...",Skiouros,birds,0.90,response_query_influence,"[0.658203125, 5.796875, -4.1171875, -0.5361328..."
67,Who played the character Mrs. Merton on TV?,Caroline Hook,Dawn French,0.99,response_query_influence,"[28.578125, 3.28125, -1.0787704433333545e-24, ..."
68,Summer Olympic sports are divided into categor...,Freestyle gymnastics,cycling,0.90,response_query_influence,"[4.71875, -0.49609375, 24.328125, 8.796875, -0..."


In [18]:
 [key for key in results.columns if "_query" not in key]

['question', 'target', 'response', 'confidence']

In [17]:
results

,question,target,response,confidence,question_query,response_query,confidence_query,qa_query,pc_query,confidence_query_influence,response_query_influence
0,"If ultimate means last, what word means second...",Penult,penultimate,0.99,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[-0.54931640625, 4.2265625, 14.1875, 1.2246093...","[2.04296875, 9.7265625, -2.326171875, 7.097656..."
1,"""Who was the villain in the movie """"Thor""""?""",Loptr,Loki,0.95,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[5.875, 0.3681640625, 5.84765625, 4.12890625, ...","[22.046875, -2.3359375, 8.84375, 1.115234375, ..."
2,In which U.S. State is the University of Harva...,Masachusetts,Massachusetts,0.10,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[4.6796875, 8.5234375, 8.6484375, 12.4765625, ...","[1.8955078125, 1.45703125, 2.197265625, 3.2128..."
3,With what group are Bobby Seale and Huey P. Ne...,Black Panther Party,black panthers,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0009679794311523438, 2.80859375, 2.43164062...","[13.3203125, -10.0234375, 0.00457763671875, 0...."
4,"What is the name of 11 Roman Emperors, the fir...",Byzantine Constantine,Byzantine Emperors,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.00033974647521972656, 3.787109375, 2.103515...","[-1.01953125, 7.66015625, -26809761792.0, 3.07..."
5,Erotology is the study of what?,The word sex,sexual practices and customs,0.90,[],[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0007748603820800781, 3.2890625, 2.544921875...","[-2.924177953162028e-27, 4.5777618232563124e-41]"
6,"Grizzled Giant, Southern Flying and Provost’s ...",Skiouros,birds,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0014677047729492188, 1.7880859375, 2.763671...","[0.658203125, 5.796875, -4.1171875, -0.5361328..."
7,Who played the character Mrs. Merton on TV?,Caroline Hook,Dawn French,0.99,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[-0.0908203125, 4.6484375, 15.90625, 0.7788085...","[28.578125, 3.28125, -1.0787704433333545e-24, ..."
8,Summer Olympic sports are divided into categor...,Freestyle gymnastics,cycling,0.90,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[0.0014629364013671875, 0.62451171875, 2.05273...","[4.71875, -0.49609375, 24.328125, 8.796875, -0..."
9,The Orinoco river meets the sea on which count...,ISO 3166-1:VE,venezuela,0.99,[],"[{'_index': 'tulu-3', '_id': 'training_data/tu...","[{'_index': 'tulu-3', '_id': 'training_data/tu...",[],[],"[-0.66845703125, 4.7265625, 14.53125, 1.693359...","[2.986328125, -1.5947265625, -96.3125, -0.8222..."


In [7]:
ds = load_dataset("yuxixia/triviaqa-test-tulu3-query", split="test")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

triviaqa-test-tulu3-query.csv:   0%|          | 0.00/92.0M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [29]:
[ key for key in ds.features if "_query" in key]

['question_query',
 'response_query',
 'confidence_query',
 'qa_query',
 'pc_query']

In [8]:
results

Dataset({
    features: ['question', 'target', 'response', 'confidence', 'question_query', 'response_query', 'confidence_query', 'qa_query', 'pc_query', 'confidence_query_influence', 'response_query_influence'],
    num_rows: 10
})

In [20]:
n_batches = 1500  % (10000 // 1000)
n_batches 

0

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-2-1124-7B-SFT")
model = AutoModelForCausalLM.from_pretrained("allenai/OLMo-2-1124-7B-SFT").to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# def preprocess_tulu_confidence(x):
#     question = x['question'][0].strip()
#     answer = x['response'][0]

#     messages = [{"role": "user", "content": f"Answer the question, give ONLY the answer, no other words or explanation:\n\n" + question},
#                 {"role": "assistant", "content":answer},
#                 {"role": "user", "content": f"Provide the probability that your answer is correct. Give ONLY the probability between 0.0 and 1.0, no other words or explanation."}]

#     return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)



ds = load_dataset("yuxixia/triviaqa-test-tulu3-query", split="test")


ds_tokenized = ds.with_transform(lambda x : )
# ds_tokenized = dataset.with_transform(
#     lambda x : tokenizer(
#         preprocess_tulu_confidence(x),
#         return_special_tokens_mask=False,
#         truncation=True,
#         padding="max_length",
#         max_length=4096,
#         return_tensors="pt"))

In [5]:

# def preprocess_triviaqa(x):
#     question = x['question'][0].strip()
#     answer = x['response'][0]

#     messages = [{"role": "user", "content": f"Answer the question, give ONLY the answer, no other words or explanation:\n\n" + question},
#                 {"role": "assistant", "content":answer},
#                 {"role": "user", "content": f"Provide the probability that your answer is correct. Give ONLY the probability between 0.0 and 1.0, no other words or explanation."}]

#     return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    


# tokenizer.pad_token_id = model.config.eos_token_id    

# ds = load_dataset("yuxixia/triviaqa-test-tulu3-query", split="test")
# ds_tokenized = ds.with_transform(lambda x : tokenizer([preprocess_triviaqa(x)],return_special_tokens_mask=False, truncation=True, padding="max_length", max_length=4096,return_tensors="pt"))


In [78]:
len(ds[0]["confidence_query"])

10

In [ ]:
ds = load_dataset("yuxixia/triviaqa-test-tulu3-query", split="test")

def flatten(x, key):
    x[key] =[list(row.values())[-1]["text"] for row in eval(x[key])]
    return x


for key in test_cols:
    ds = ds.map(lambda x: flatten(x,key=key), num_proc=100) 
ds[0]

{'question': 'If ultimate means last, what word means second-to-last?',
 'target': 'Penult',
 'response': 'penultimate',
 'confidence': 0.99,
 'question_query': '[]',
 'response_query': ["Develop a Python algorithm that identifies the penultimate highest value within an unordered list of distinct positive integers. Additionally, optimize the algorithm for time complexity and consider edge-case scenarios such as one-element lists. \narr = [1, 3, 4, 5, 9, 10, 11] Here's a Python code solution:\n\n```python\ndef penultimate_highest(numbers):\n    if len(numbers) < 2:\n        # If list is empty, there is no penultimate element\n        # If list only contains one number, there is no penultimate element\n        return 'The list contains less than 2 elements'\n    \n    highest = max(numbers[0], numbers[1])\n    penultimate = min(numbers[0], numbers[1])\n    \n    for num in numbers[2:]:\n        if num > highest:\n            penultimate = highest\n            highest = num\n        elif 

In [63]:
for row in ds:
    for test_col in test_cols:
        row[test_col+"_influence"] = [1 for example in  row[test_col]]
        


In [ ]:
[row[test_col+"_influence"] = [1 for example in  row[test_col]]]

'triviaqa-test-tulu3-query'

In [1]:
results = load_dataset("./results_confidence/triviaqa-test-tulu3-query")

NameError: name 'load_dataset' is not defined

In [ ]:
for i in range(0,10):
        outputs = model.generate(**ds_tokenized[i:i+1].to(device), 
        tokenizer=tokenizer,
        max_new_tokens=20, 
        do_sample=False, 
        # num_beams=1, 
        eos_token_id=tokenizer.eos_token_id,
        # output_scores=True, 
        )
        print("----")
        print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
        print("----------------------")

In [ ]:
ds_tokenized[0:10]